In [1]:
import numpy as np
import pandas as pd
from PIL import Image

In [2]:
image =Image.open("/kaggle/input/test-photo/Official Photo.jpg").convert("L")
image =image.resize((5,5))
img_array = np.array(image, dtype=float) / 255
print(img_array)
print(img_array.shape)

[[0.14509804 0.67058824 0.79607843 0.70196078 0.69411765]
 [0.17647059 0.47058824 0.60784314 0.3254902  0.34117647]
 [0.51764706 0.35294118 0.48235294 0.36862745 0.37254902]
 [0.62352941 0.51764706 0.60392157 0.28627451 0.4       ]
 [0.84313725 0.70980392 0.61568627 0.56470588 0.51764706]]
(5, 5)


In [3]:
def Conv2d(input, kernel):
    h, w = input.shape
    kh, kw = kernel.shape
    output_h = h - kh + 1
    output_w = w - kw + 1
    output = np.zeros((output_h, output_w))
    
    for i in range(output_h):
        for j in range(output_w):
            region = input[i:i+kh, j:j+kw]
            output[i, j] = np.sum(region * kernel)
    return output

In [4]:
def relu(z):
     return np.maximum(0,z)

In [5]:
def relu_derivative(x):
    return (x > 0).astype(float)

In [6]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [7]:
def sigmoid_derivative(x):
    s = sigmoid(x)
    return s * (1 - s)

In [8]:
def dense(input_vec,weights, bias):
    return np.dot(weights,input_vec)+bias

In [9]:
def mse_loss(pred,target):
    return np.mean((pred-target)**2)

In [10]:
def mse_grad(pred, target):
    return 2 * (pred - target) / len(pred)

In [11]:
def gradient_descent(weights, bias, grad_output, input_vec, lr):
    weights -= lr * np.outer(grad_output, input_vec)
    bias -= lr * grad_output
    return weights, bias

In [23]:
# Sample 3x3 kernel (edge detection)
kernel = np.array([[1, 0, -1],
                   [1, 0, -1],
                   [1, 0, -1]], dtype=np.float32)

# Conv ➝ ReLU
conv_output = Conv2d(img_array, kernel)
relu_output1 = relu(conv_output)

# Flatten for dense input
flattened = relu_output1.flatten()

# Dense ➝ ReLU ➝ Sigmoid
output_size = 2
input_size = len(flattened)
weights = np.random.randn(output_size, input_size)
bias = np.random.randn(output_size)

dense_output = dense(flattened, weights, bias)
relu_output2 = relu(dense_output)
final_output = sigmoid(relu_output2)  # squash values to [0, 1]

target = np.array([0.8, 0.2])
loss = mse_loss(final_output, target)

print("Conv Output:\n", conv_output)
print("ReLU 1 Output:\n", relu_output1)
print("Flattened Input:\n", flattened)
print("Dense Output:\n", dense_output)
print("ReLU 2 Output:\n", relu_output2)
print("Final Sigmoid Output (Prediction):\n", final_output)
print("Target:", target)
print("MSE Loss:", loss)

Conv Output:
 [[-1.04705882  0.09803922  0.47843137]
 [-0.37647059  0.36078431  0.58039216]
 [ 0.28235294  0.36078431  0.41176471]]
ReLU 1 Output:
 [[0.         0.09803922 0.47843137]
 [0.         0.36078431 0.58039216]
 [0.28235294 0.36078431 0.41176471]]
Flattened Input:
 [0.         0.09803922 0.47843137 0.         0.36078431 0.58039216
 0.28235294 0.36078431 0.41176471]
Dense Output:
 [ 0.03488176 -0.67794786]
ReLU 2 Output:
 [0.03488176 0.        ]
Final Sigmoid Output (Prediction):
 [0.50871956 0.5       ]
Target: [0.8 0.2]
MSE Loss: 0.0874221485413032


In [26]:
kernel = np.array([[1, 0, -1],
                   [1, 0, -1],
                   [1, 0, -1]], dtype=np.float32)
output_size = 2
input_size = len(flattened)
weights = np.random.randn(output_size, input_size)
bias = np.random.randn(output_size)
target = np.array([0.8, 0.2])
lr = 0.001
epochs = 20
for epoch in range(epochs):
    # Forward
    conv_output = Conv2d(img_array, kernel)
    relu_output1 = relu(conv_output)
    flattened = relu_output1.flatten()
    dense_output = dense(flattened, weights, bias)
    relu_output2 = relu(dense_output)
    final_output = sigmoid(relu_output2)

    # Loss
    loss = mse_loss(final_output, target)

    # Backprop
    grad_loss = mse_grad(final_output, target)
    grad_sigmoid = sigmoid_derivative(relu_output2)
    grad_relu2 = relu_derivative(dense_output)
    total_grad = grad_loss * grad_sigmoid * grad_relu2

    weights, bias = gradient_descent(weights, bias, total_grad, flattened, lr)

    print(f"Epoch {epoch+1}, Loss: {loss:.6f}")

Epoch 1, Loss: 0.048199
Epoch 2, Loss: 0.048199
Epoch 3, Loss: 0.048199
Epoch 4, Loss: 0.048199
Epoch 5, Loss: 0.048199
Epoch 6, Loss: 0.048198
Epoch 7, Loss: 0.048198
Epoch 8, Loss: 0.048198
Epoch 9, Loss: 0.048198
Epoch 10, Loss: 0.048198
Epoch 11, Loss: 0.048198
Epoch 12, Loss: 0.048198
Epoch 13, Loss: 0.048197
Epoch 14, Loss: 0.048197
Epoch 15, Loss: 0.048197
Epoch 16, Loss: 0.048197
Epoch 17, Loss: 0.048197
Epoch 18, Loss: 0.048197
Epoch 19, Loss: 0.048197
Epoch 20, Loss: 0.048196
